In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 📊 Hedged Risk Portfolio\n",
    "\n",
    "🚀 Финансово-инженерный проект по управлению портфелем с многоуровневым хеджем\n",
    "\n",
    "## Состав портфеля:\n",
    "- 95% базовые активы: BTC, ETH, XRP, Gold (XAU/USD)\n",
    "- 3% защитные пут-опционы (BTC и ETH)\n",
    "- 2% волатильность-хедж через адаптивный strangle на BTC\n",
    "\n",
    "## Методы:\n",
    "- Black-Scholes (BS) модель опционов\n",
    "- Rolling 30-дневные опционы\n",
    "- VIX proxy на основе rolling volatility BTC\n",
    "- VaR, Expected Shortfall, Max Drawdown\n",
    "- Визуализация PnL, волатильности, веса хеджа\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📦 Импорт библиотек\n",
    "import yfinance as yf\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "from scipy.stats import norm\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🔧 Настройка параметров\n",
    "assets = ['BTC-USD', 'ETH-USD', 'XRP-USD', 'XAUUSD=X']\n",
    "weights = {'BTC-USD': 0.30, 'ETH-USD': 0.25, 'XRP-USD': 0.15, 'XAUUSD=X': 0.25}\n",
    "hedged_assets = ['BTC-USD', 'ETH-USD']\n",
    "sigmas = {'BTC-USD': 0.70, 'ETH-USD': 0.80}\n",
    "r = 0.05\n",
    "holding_days = 30"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📥 Загрузка данных с Yahoo Finance\n",
    "data = yf.download(assets, start='2020-01-01', end='2023-12-31')['Adj Close']\n",
    "returns = np.log(data / data.shift(1)).dropna()\n",
    "prices = data.loc[returns.index]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ⚙️ Black-Scholes функции\n",
    "def bs_call(S, K, T, r, sigma):\n",
    "    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))\n",
    "    d2 = d1 - sigma*np.sqrt(T)\n",
    "    return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)\n",
    "\n",
    "def bs_put(S, K, T, r, sigma):\n",
    "    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))\n",
    "    d2 = d1 - sigma*np.sqrt(T)\n",
    "    return K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)"
   ]
  }
 ]
}



{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📈 Rolling VIX proxy (волатильность BTC)\n",
    "rolling_vol = returns['BTC-USD'].rolling(window=30).std() * np.sqrt(365)\n",
    "vix_z = (rolling_vol - rolling_vol.rolling(60).mean()) / rolling_vol.rolling(60).std()\n",
    "\n",
    "def get_strangle_weight(z):\n",
    "    if z >= 1:\n",
    "        return 0.03\n",
    "    elif z >= 0:\n",
    "        return 0.02\n",
    "    else:\n",
    "        return 0.01"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🔁 Rolling PnL расчёты\n",
    "dates = returns.index[::holding_days]\n",
    "plain_pnl = pd.Series(0, index=returns.index)\n",
    "hedge_puts = pd.Series(0, index=returns.index)\n",
    "hedge_strangle = pd.Series(0, index=returns.index)\n",
    "\n",
    "for i in range(len(dates) - 1):\n",
    "    t0, t1 = dates[i], dates[i + 1]\n",
    "    slice_prices = prices.loc[t0:t1]\n",
    "    T = (t1 - t0).days / 365\n",
    "\n",
    "    for ticker, w in weights.items():\n",
    "        S0 = prices.loc[t0, ticker]\n",
    "        S_series = slice_prices[ticker]\n",
    "        spot_pnl = (S_series - S0) * (w * 0.95)\n",
    "        plain_pnl.loc[t0:t1] += spot_pnl\n",
    "\n",
    "        if ticker in hedged_assets:\n",
    "            K_put = S0 * 0.85\n",
    "            sigma = sigmas[ticker]\n",
    "            put_price = bs_put(S0, K_put, T, r, sigma)\n",
    "            put_payoff = np.maximum(K_put - S_series, 0)\n",
    "            put_pnl = (put_payoff - put_price) * (0.03 / len(hedged_assets))\n",
    "            hedge_puts.loc[t0:t1] += put_pnl"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🔥 Strangle хедж\n",
    "    z = vix_z.loc[t0] if t0 in vix_z else 0\n",
    "    strangle_weight = get_strangle_weight(z)\n",
    "\n",
    "    S_btc = prices.loc[t0, 'BTC-USD']\n",
    "    S_btc_series = slice_prices['BTC-USD']\n",
    "    Kp, Kc = S_btc * 0.85, S_btc * 1.15\n",
    "    sigma_btc = sigmas['BTC-USD']\n",
    "\n",
    "    put_cost = bs_put(S_btc, Kp, T, r, sigma_btc)\n",
    "    call_cost = bs_call(S_btc, Kc, T, r, sigma_btc)\n",
    "    total_cost = put_cost + call_cost\n",
    "\n",
    "    put_payoff = np.maximum(Kp - S_btc_series, 0)\n",
    "    call_payoff = np.maximum(S_btc_series - Kc, 0)\n",
    "    strangle_pnl = (put_payoff + call_payoff - total_cost) * strangle_weight\n",
    "    hedge_strangle.loc[t0:t1] += strangle_pnl"
   ]
  }
 ]
}


{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📊 Сборка итогового портфеля\n",
    "total_pnl = plain_pnl + hedge_puts + hedge_strangle\n",
    "total_pnl = total_pnl.dropna()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📉 Расчёт метрик риска\n",
    "def compute_var_es(pnl_series, confidence=0.95):\n",
    "    var = np.percentile(pnl_series, (1 - confidence) * 100)\n",
    "    es = pnl_series[pnl_series <= var].mean()\n",
    "    return var, es\n",
    "\n",
    "def max_drawdown(cum_pnl):\n",
    "    peak = cum_pnl.expanding(min_periods=1).max()\n",
    "    dd = cum_pnl / peak - 1\n",
    "    return dd.min()\n",
    "\n",
    "var, es = compute_var_es(total_pnl)\n",
    "cum_pnl = total_pnl.cumsum()\n",
    "dd = max_drawdown(cum_pnl)\n",
    "\n",
    "print(f\"📉 VaR (95%): {var:.2%}\")\n",
    "print(f\"📉 Expected Shortfall (95%): {es:.2%}\")\n",
    "print(f\"📉 Max Drawdown: {dd:.2%}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📈 Кумулятивная прибыль\n",
    "plt.figure(figsize=(10,6))\n",
    "plt.plot(cum_pnl, label='Итоговый портфель')\n",
    "plt.title(\"💰 Кумулятивный PnL (2020–2023)\")\n",
    "plt.ylabel(\"PNL ($)\")\n",
    "plt.grid(True)\n",
    "plt.legend()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📊 Распределение PnL\n",
    "plt.figure(figsize=(10,6))\n",
    "plt.hist(total_pnl, bins=60, alpha=0.7)\n",
    "plt.axvline(var, color='red', linestyle='--', label=f\"VaR 95%: {var:.2%}\")\n",
    "plt.title(\"Распределение дневного PnL\")\n",
    "plt.xlabel(\"PnL\")\n",
    "plt.ylabel(\"Частота\")\n",
    "plt.grid(True)\n",
    "plt.legend()\n",
    "plt.show()"
   ]
  }
 ]
}


{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📉 Rolling VIX proxy\n",
    "plt.figure(figsize=(10, 5))\n",
    "plt.plot(rolling_vol, label='30-day Annualized Volatility (BTC)')\n",
    "plt.title(\"📊 VIX Proxy: Rolling Historical Volatility of BTC\")\n",
    "plt.grid(True)\n",
    "plt.legend()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🧠 Z-score по волатильности\n",
    "plt.figure(figsize=(10, 5))\n",
    "plt.plot(vix_z, label='VIX Z-score (BTC)', color='purple')\n",
    "plt.axhline(0, linestyle='--', color='gray')\n",
    "plt.axhline(1, linestyle='--', color='red', label='High Vol Threshold')\n",
    "plt.axhline(-1, linestyle='--', color='green', label='Low Vol Threshold')\n",
    "plt.title(\"Z-score of BTC Volatility\")\n",
    "plt.grid(True)\n",
    "plt.legend()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📘 Выводы:\n",
    "- Rolling VIX proxy позволяет гибко управлять затратами на хеджирование\n",
    "- Protective puts эффективно ограничивают убытки при падении рынка\n",
    "- Strangle даёт экспозицию на резкие движения и рост волатильности\n",
    "- Итоговый портфель показывает устойчивость: сниженный VaR, меньший drawdown, и сглаженный PnL\n",
    "\n",
    "📈 Всё это делает стратегию подходящей для крипто-периодов высокой неопределённости\n"
   ]
  }
 ]
}

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 📤 Экспорт отчёта в PDF\n",
    "\n",
    "Если ты работаешь в Jupyter Lab или VS Code:\n",
    "\n",
    "```bash\n",
    "jupyter nbconvert --to pdf notebooks/hedged_portfolio.ipynb\n",
    "```\n",
    "\n",
    "Или через меню → File → Download as → PDF.\n",
    "\n",
    "_Убедись, что установлен `TeX` (например, `MiKTeX` или `TeXLive`)_"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "## 🎉 Конец ноутбука\n",
    "Спасибо за работу с этой моделью! Хочешь добавить:\n",
    "- implied volatility вместо исторической?\n",
    "- опционы на другие активы (ETH, XRP)?\n",
    "- или rolling VaR с Horizon = 5 дней?\n",
    "\n",
    "_Открыт к расширениям!_"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}




